In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from models import MuseGAN
from utils import load_music

In [ ]:
# run params
SECTION = 'compose'
RUN_ID = '0002'
DATA_NAME = 'chorales'
FILENAME = 'Jsb16thSeparated.npz'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))
    os.mkdir(os.path.join(RUN_FOLDER, 'samples'))

mode = 'build'  # ' 'load' #

In [ ]:
BATCH_SIZE = 64
n_bars = 2
n_steps_per_bar = 16
n_pitches = 84
n_tracks = 4

data_binary, data_ints, raw_data = load_music(DATA_NAME, FILENAME, n_bars, n_steps_per_bar)
data_binary = np.squeeze(data_binary)

In [ ]:
gan = MuseGAN(
    input_dim=data_binary.shape[1:], 
    critic_learning_rate=0.001, 
    generator_learning_rate=0.001, 
    optimizer='adam', 
    grad_weight=10, 
    z_dim=32, 
    batch_size=BATCH_SIZE, 
    n_tracks=n_tracks, 
    n_bars=n_bars, 
    n_steps_per_bar=n_steps_per_bar, 
    n_pitches=n_pitches
)

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(RUN_FOLDER)


In [ ]:
gan.chords_temp_network.summary()

In [ ]:
gan.bar_gen[0].summary()

In [ ]:
gan.generator.summary()

In [ ]:
gan.critic.summary()

In [ ]:
EPOCHS = 6000
PRINT_EVERY_N_BATCHES = 100

gan.epoch = 0
gan.train(
    data_binary,
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    run_folder=RUN_FOLDER, 
    print_every_n_batches=PRINT_EVERY_N_BATCHES
)


In [ ]:
fig = plt.figure()
plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.plot([x[1] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot(gan.g_losses, color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, len(gan.d_losses))
# plt.ylim(0, 2)
plt.savefig(os.path.join(RUN_FOLDER, 'images/loss.png'))

plt.show()
